### OPERATORNORM WITHOUT ADJOINT 

- Creator:  Jonas Bresch, M.Sc
- Date: February 14rd, 2025

In [240]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc

import imp

In [241]:
import adjointfreenorm
imp.reload(adjointfreenorm)

<module 'adjointfreenorm' from '/Users/jonasbresch/Desktop/operator_missmatch/finale/adjointfreenorm.py'>

### DATA

In [246]:
#
# random matrix
#

A = np.random.randn(10, 50)
V = np.random.randn(50, 10)

In [232]:
#
# random orthogonal matrix
#

n = 100
H = np.random.randn(n, n)
Q, R = sc.linalg.qr(H)
Aort = Q.dot(Q.T)

In [233]:
#
# two examples, giving the illustration in Fig.2
#   -> Alg.2 (two stepsizes) converges in exact one iteration, every time
#   -> Alg.1 (one stepsizes) does NOT converge necessarly in one iteration.
#

A = np.array([[1,0], [0,0]])
A = np.array([[1,0], [0,1], [0,0]])

### Algorithms
    - MatFreeAdjNorm for computing ||A|| from Bresch, Lorenz, Schneppe, Winkler (2024)
    - MatFreeAdjOpNormDouble for computing ||A - V|| with two stepsizes
    - MatFreeAdjOpNorm for computing ||A - V|| with one stepsize

In [247]:
vv, optvv, vvsol, vvval, listtau, lista, listerror = adjointfreenorm.MatFreeAdjNorm(
    A=A, 
    # A=Aort,
    iter=100000, 
    eps=1e-10, 
    nor=1, 
    show=0)

iter. 	| func-value 	| residuum  	| sing-vec-error
3609 	|  8.150e+01 	|  2.274e-13 	|  2.776e-15
||A|| =  9.027786305789062


In [248]:
u, v, vu_sol, optu, optv, val, listtau_uv, listabcd_uv, listbc_uv, listerror_uv, listapprox_norm, listapprox_a, listapprox_a_min, listapprox_sig = adjointfreenorm.MatFreeAdjOpNormDouble(
    A=A, 
    # V=V,        # optional argument, if it is left free, we compute just ||A||
    # V=A.T, 
    iter=100000, 
    eps=1e-7, 
    show=0)

iter. 	| func-value 	| residuum 	| sing-vec-error
2985 	|  9.028e+00 	|  2.659e-11 	|  1.044e-14
||A|| =  9.027786305762495


In [249]:
u_o, v_o, vu_sol_o, optu_o, optv_o, vvvual_o, listtau_uv_o, lista_uv_o, listerror_uv_o = adjointfreenorm.MatFreeAdjOpNorm(
    A=A, 
    # V=V,        # optional argument, if it is left free, we compute just ||A||
    # V=A.T, 
    iter=200000, 
    eps=1e-11,
    show=0)

iter 	| func-value 	| residuum  	| sing-vec-error
19999 	|  9.028e+00 	|  1.421e-13 	|  4.240e-08
||A|| =  9.027786305788931
